## Анализ поведения клиентов и оценка каким клиентам предложить1 из новых тарифов: «Смарт» или «Ультра».

In [2]:
import warnings
warnings.filterwarnings("ignore") 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier  
from sklearn.linear_model import LogisticRegression  
from sklearn.dummy import DummyClassifier 

## Обзор данных

описание данных:
1. сalls — количество звонков
2. minutes — суммарная длительность звонков в минутах,
3. messages — количество sms-сообщений,
4. mb_used — израсходованный интернет-трафик в Мб,
5. is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [6]:
#data_frame=pd.read_csv('users_behavior.csv')
data_frame=pd.read_csv('/datasets/users_behavior.csv')

data_frame.info()
data_frame.head(10) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## Предподгатовка

In [7]:
null_data_columns=data_frame.isna().sum() 
columns_data_old=list(data_frame.columns) 
display(null_data_columns)
null_data_columns=list(null_data_columns) 

print('Доля пропусков в столбце:') 
for i in range(len(columns_data_old)): 
    rezul=1-(len(data_frame)-null_data_columns[i])/len(data_frame)
    print(columns_data_old[i], '=', f'{rezul:.2%}' )
  

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Доля пропусков в столбце:
calls = 0.00%
minutes = 0.00%
messages = 0.00%
mb_used = 0.00%
is_ultra = 0.00%


In [8]:
features =data_frame.drop(['is_ultra',], axis=1) 
target =data_frame['is_ultra']  
# разделяем на выборку для обучения и тестирования, 25% данных для валидации (теста)
features_train, features_data, target_train, target_data = train_test_split(features, target, test_size=0.5, random_state=2504)
features_test, features_valid, target_test, target_valid = train_test_split( features_data, target_data, test_size=0.5, random_state=2504)

## Исследование разных моделей машинного обучения

### Дерево решений

In [9]:
%%time
best_model_DecisionTree = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=2504, max_depth=depth).fit(features_train,target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score( target_valid, predictions) 
    if result > best_result:
        best_model_DecisionTree = model
        best_result = result
print("Точность лучшей модели:", best_result)
print("Параметры лучшей модели ",best_model_DecisionTree) 
                                               

Точность лучшей модели: 0.8072139303482587
Параметры лучшей модели  DecisionTreeClassifier(max_depth=5, random_state=2504)
CPU times: user 93.9 ms, sys: 0 ns, total: 93.9 ms
Wall time: 98.2 ms


Тест модели

In [10]:
predictions=best_model_DecisionTree.predict(features_test)
result = accuracy_score(target_test, predictions) 
print("Точность лучшей модели на тестовых данных:",result)

Точность лучшей модели на тестовых данных: 0.7932752179327521


**Вывод**

Лучший результат показала модель с параметрами, глубина обучения 5, а точность модели составила: 0.793 на тестовых данных

### Случайный лес

In [11]:
%%time
best_model_RandomFores = None
best_result = 0
for estima in range(10, 101, 5):
    for depth in range(1, 16):
       
        model = RandomForestClassifier(random_state=2504, max_depth=depth, n_estimators=estima).fit(features_train,target_train)
        predictions = model.predict(features_valid)
        result = accuracy_score( target_valid, predictions) 
        if result > best_result:
            best_model_RandomFores = model
            best_result = result
print("Точность лучшей модели:", best_result)
print("Параметры лучшей модели ",best_model_RandomFores)


Точность лучшей модели: 0.8171641791044776
Параметры лучшей модели  RandomForestClassifier(max_depth=9, n_estimators=10, random_state=2504)
CPU times: user 59.2 s, sys: 286 ms, total: 59.4 s
Wall time: 1min


Тест модели

In [12]:
predictions=best_model_RandomFores.predict(features_test)
result = accuracy_score(target_test, predictions) 
print("Точность лучшей модели на тестовых данных:",result)

Точность лучшей модели на тестовых данных: 0.7895392278953923


**Вывод**

Лучший результат показала модель с параметрами, глубина обучения 10, количество деревьев 60, а точность модели составила: 0.789 на тестовых данных

### Логическая регресия

In [13]:
%%time
best_model_LogcRegres = None
best_result = 0
model = LogisticRegression(random_state=2504).fit(features_train,target_train)   
predictions = model.predict(features_valid) 
result = accuracy_score( target_valid, predictions) 
print("Точность лучшей модели:", result)
print("Параметры лучшей модели ",model)

Точность лучшей модели: 0.7761194029850746
Параметры лучшей модели  LogisticRegression(random_state=2504)
CPU times: user 42.7 ms, sys: 34 µs, total: 42.8 ms
Wall time: 40.4 ms


Тест модели

In [15]:
predictions=model.predict(features_test)
result = accuracy_score(target_test, predictions) 
print("Точность лучшей модели на тестовых данных:",result)

Точность лучшей модели на тестовых данных: 0.7334993773349938


**Вывод**

1. Для логической регрессии точность модели составила: 0.733, что ниже порогового значения в 0.75. Таким образом данная модель не подходит. 
2. Лучшие результаты точности (выше 0.75) показали модели дерево решений и случайный лес.

### Сравнение скорости обучения 

In [16]:
%%time
model = DecisionTreeClassifier(random_state=2504, max_depth=5).fit(features_train,target_train)
predictions = model.predict(features_valid) 
result = accuracy_score( target_valid, predictions) 
print("Точность лучшей модели:", result)
print("Параметры лучшей модели ",best_model_DecisionTree) 

Точность лучшей модели: 0.8072139303482587
Параметры лучшей модели  DecisionTreeClassifier(max_depth=5, random_state=2504)
CPU times: user 14.1 ms, sys: 26 µs, total: 14.1 ms
Wall time: 11.7 ms


In [17]:
%%time
model = RandomForestClassifier(random_state=2504, max_depth=10, n_estimators=60).fit(features_train,target_train) 
predictions = model.predict(features_valid) 
result = accuracy_score( target_valid, predictions) 
print("Точность лучшей модели:", result)
print("Параметры лучшей модели ",best_model_RandomFores)


Точность лучшей модели: 0.7972636815920398
Параметры лучшей модели  RandomForestClassifier(max_depth=9, n_estimators=10, random_state=2504)
CPU times: user 245 ms, sys: 8 µs, total: 245 ms
Wall time: 242 ms


**Вывод**

Модель "дерево решений" имеет точность предсказания меньше на 0,016, чем модель "случейного леса", но скорость обучения модели "случайного леса" примерно в 20 раз меньше, чем у модели "дерева решений". Таким образом логичнее использовать модель "дерева решений".

### Сравнение моделей 

In [18]:
model_dummy =  DummyClassifier(strategy='most_frequent').fit(features_train,target_train) 
predictions = model_dummy.predict(features_valid) 
result = accuracy_score( target_valid, predictions) 
print("Точность лучшей модели:", result)
print("Параметры лучшей модели ",model_dummy)

Точность лучшей модели: 0.7338308457711443
Параметры лучшей модели  DummyClassifier(strategy='most_frequent')
